# Build a Chatbot with Neural Network 

We've discovered how to build a chatbot with cosine similarity. Now, let's explore how we might build one with neural network!

We will create our training data, train a neural network with them, then use the trained model to make our chatbot. 

First, we will install required libraries. Uncomment the few blocks below only if you do not have the libraries installed. 

In [5]:
!pip install numpy scipy
!pip install scikit-learn
!pip install pillow
!pip install h5py


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
!pip install tensorflow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


Requested tensorflow-gpu from https://files.pythonhosted.org/packages/8a/45/fa31ced1db38f9424f262dfbf35747fe5378b5c808cecb373c8cb8e515d3/tensorflow-gpu-2.12.0.tar.gz has invalid metadata: Expected end or semicolon (after name and no valid version specifier)
    python_version>"3.7"
                  ^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: 2.12.0)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow-gpu


In [10]:
!pip install keras


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hisha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# 1. Install Libraries

Firstly, we will install libraries needed for this neural network powered chatbot. 
Keras is a machine learning library which utilizes tensorflow (another lower level machine learning library) at the backend. This makes it easier for us to deploy deep neural network for this purpose. 

In [11]:
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re

# 2. Input training data

We will first include the following training data for our chatbot:
1. X represent the different possible inputs that users might enter
2. Y represent the intent of the inputs

In [13]:
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

In [14]:
print(len(X))

10


In [16]:
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

In [17]:
print(len(Y))

10


Notice that there are several different sentences that have similar intent. Here, we are only having 3 intents, but you can add as many as you want for your project!

This is the way our chatbot will work:
1. From the input sentence, we will identify the intent using our trained AI model.
2. For each intent, we have a prepared response. 

For example, if we identify that the intent of the input is for a greeting, we might ask the chatbot to reply with a greeting as well, something like 'hi' or 'how are you doing?'

We will use machine learning to create a model that can classify input sentence into different intents. 
We make it as follows:

1. We create a training data (X and Y above) which contains a list of sentences and their intents.
2. Use the training data to train a classifier. 
3. Vectorize input sentences and use classifier to determine intent. 

# 3. Text processing

As usual, we will start with text processing. Do you remember the process?

## 3.1 Remove non alphanumeric characters

In [18]:
def remove_non_alpha_numeric_characters(sentence):
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

In [19]:
def preprocess_data(X):
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(
        sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ',
                data_point) for data_point in X]
    return X

In [20]:
X = preprocess_data(X)

vocabulary = set()
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)

## Create document vectors

In [21]:
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]

In [22]:
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

# 4. Create training data and test data

In [23]:
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

Print and check the data you are using for training and test data

In [24]:
print (y_test)

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1]]


In [25]:
print(X_train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [26]:
print(len(X_train))

10


In [27]:
y_train

[[1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1]]

What does y_train represent? Do you understand the array shown above?

# 5. Model training

Now we will use the training data to train our neural network.

In [28]:
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',
                input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
model.summary()

C:\Users\hisha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,155 (4.51 KB)

 Trainable params: 1,155 (4.51 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(learning_rate=0.01,
                            momentum=0.9, nesterov=True))
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=16)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step - loss: 1.1569
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.1447
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.1304
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1164
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1048
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.0968
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 1.0923
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 1.0907
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.0909
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.0916
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.0919
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.0911
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.0892
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.0862
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.0826
Epoch 16/100
1/1 ━

## List down predictions

In [31]:
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


# Model Evaluation

Let's evaluate our model now. We will compare the prediction made by the model and our test data:

In [32]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 8
Total: 10


# Testing the chatbot

Let's test the chatbot now! We will input a sentence, and then see what class is predicted by the neural network:

In [ ]:
while True:
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    print (classes[argmax(prediction)])

Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
greeting
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
busy
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
busy
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
bye
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
busy
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
busy
Enter a sentence


Realize that you can't stop the chatbot? You'll have to add the exit command later (see the previous notebook to find out how to do it. 

For now, simply press the stop button (interrupt button) above to stop the chatbot. 

Try it! press the stop button, and try typing something onto the box. 

# Challenge

We have successfully use neural network to map our input to conversation intent. 
Your challenge is to link the conversation intent to a particular response that the chatbot will say. 
For example, if the conversation intent is 'greeting', get your chatbot to say a greeting as well!

In [33]:
while True:
    print("Enter a sentence")
    sentence = input()
    prediction = model.predict(np.array([encode_sentence(sentence)]))
    if classes[argmax(prediction)] == 'bye':
        print('Goodbye! See you next time!')
        break
    elif classes[argmax(prediction)] == 'busy':
        print('ah... all the best!')
    elif classes[argmax(prediction)] == 'greeting':
        print('Hi!')

Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Hi!
Enter a sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Goodbye! See you next time!


### Great job! You've successfully created a simple chatbot with neural network! How might you improve the chatbot?
You can improve the chatbot by:
- Adding more training data
- Adding more intent
- Focusing on a particular topic and train the chatbot with many training data in that topic

### Resource:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/